In [ ]:
# Project #1 Juypter Notebook
# Created by Melinda Eudy, Cameron Farquhar, Markita Francis, Luis Rojas, Kelly Rose
# Project :

In [6]:
# Dependencies and Setup *******Review before final presentation and remove what is not needed*********
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress
import scipy.stats as st
from datetime import date
import json
import ctypes  # An included library with Python install.
import datetime

# Import API key
#from config import apikey

In [8]:
# Part 1 - open UCR Crime Rates by County.csv file
#----------open 37059-Codebook County Codes.csv file and merge into first file to obtain County Names 
UCR_Crime = "/Resources/UCR_Crime_Rates_by_County.csv"

# Read file and store in Pandas data frame
UCR_Crime_df = pd.read_csv(UCR_Crime)
UCR_Crime_df.head()
UCR_Crime_df.count()

FileNotFoundError: [Errno 2] No such file or directory: '/Resources/UCR_Crime_Rates_by_County.csv'

In [ ]:
UCR_Crime = ".csv"

# Read file and store in Pandas data frame
UCR_Crime_df = pd.read_csv(UCR_Crime)
UCR_Crime_df.head()
UCR_Crime_df.count()